In [43]:
import pandas as pd
import os

## Lade Daten

In [44]:
def year_to_path(year):
    path = str(year)
    files = os.listdir(os.path.join("bka", path))
    if len(files) > 0:
        return os.path.join("bka", path, files[0])

In [83]:
## 2013

df_2013 = pd.read_excel(year_to_path(2013), skiprows=6)[["Unnamed: 1", "Unnamed: 2", "Fälle"]].dropna(subset = ["Unnamed: 2"])

df_2013.rename(columns = {"Unnamed: 1" : "Art", "Unnamed: 2" : "kreis_key", "Fälle" : "crimes_2013"}, inplace = True)

df_2013 = df_2013[df_2013.Art == "Straftaten insgesamt"][["kreis_key", "crimes_2013"]]

df_2013.kreis_key = df_2013.kreis_key.astype(int)
df_2013.crimes_2013 = df_2013.crimes_2013.astype(int)

df_2013.head()

,kreis_key,crimes_2013
2,1001,8876
3,1002,24043
4,1003,24526
5,1004,11737
6,1051,7248


In [84]:
## 2014 - 2019
def read_crime_data_14_20(year):
    df = pd.read_csv(year_to_path(year), encoding = "ISO-8859-1", delimiter = ";", skiprows = 1, thousands = ",")

    df = df[df.Straftat == "Straftaten insgesamt"]

    crime_clm = f"crimes_{year}"

    df.rename(columns = {"Gemeindeschlüssel" : "kreis_key", "Anzahl erfasste Faelle" : crime_clm,  "erfasste Fälle" : crime_clm, "Gemeindeschluessel" : "kreis_key", "erfasste Faelle" :crime_clm}, inplace = True)

    df.kreis_key = df.kreis_key.astype(int)

    df = df[["kreis_key", crime_clm]]

    return df

df = df_2013.copy()

for i in range(2014, 2021):
    df2 = read_crime_data_14_20(i)
    df = df.merge(df2, on = "kreis_key", how = "outer")

df.head()

,kreis_key,crimes_2013,crimes_2014,crimes_2015,crimes_2016,crimes_2017,crimes_2018,crimes_2019,crimes_2020
0,1001,8876.0,9754.0,10828.0,9993.0,8907.0,8156.0,8617.0,8380.0
1,1002,24043.0,24380.0,26078.0,25908.0,23934.0,24621.0,22452.0,21384.0
2,1003,24526.0,24907.0,22881.0,25647.0,23614.0,22685.0,21768.0,20942.0
3,1004,11737.0,13078.0,17418.0,14524.0,9751.0,10688.0,11397.0,9453.0
4,1051,7248.0,7189.0,7345.0,7661.0,7262.0,7224.0,6786.0,6597.0


In [85]:
df.to_csv("crimes.csv")